In [1]:
]activate ../.

 Activating environment at `~/Documents/code/generators/Project.toml`


In [3]:
]st

Project EAMC v0.1.0
Status `~/Documents/code/generators/Project.toml`
  [7d9fca2a] Arpack v0.4.0
  [35d6a980] ColorSchemes v3.6.0
  [8f4d0f93] Conda v1.4.1
  [31a5f54b] Debugger v0.6.4
  [0c46a032] DifferentialEquations v6.11.0
  [31c24e10] Distributions v0.22.5
  [f6369f11] ForwardDiff v0.10.9
  [4076af6c] JuMP v0.21.1
  [0b1a1467] KrylovKit v0.4.1
  [50d2b5c4] Lazy v0.15.0
  [ee78f7c6] Makie v0.9.5
  [4d4711f2] MatrixMarket v0.3.1
  [c03570c3] Memoize v0.4.3
  [429524aa] Optim v0.20.2
  [d96e819e] Parameters v0.12.0
  [91a5bcdd] Plots v0.29.4
  [295af30f] Revise v2.5.3
  [2913bbd2] StatsBase v0.32.1
  [37e2e46d] LinearAlgebra
  [2f01184e] SparseArrays
  [10745b16] Statistics


In [16]:
using Revise
using LinearAlgebra
using MatrixMarket
using EAMC
using SparseArrays
using Plots
using Interact

LoadError: ArgumentError: Package Interact not found in current path:
- Run `import Pkg; Pkg.add("Interact")` to install the Interact package.


In [5]:
## load data
using MatrixMarket
data = cd("data/natalia191219") do 
    map(MatrixMarket.mmread, readdir())
end;

LoadError: IOError: chdir data/natalia191219: no such file or directory (ENOENT)

In [6]:
## adjust diagonal to be a proper q matrix
using LinearAlgebra
qs = data
qs = map(qs) do q
    q = q * 0.36
    q .- Diagonal(sum(q, dims=2)|>vec) # mass conservation
end;

LoadError: UndefVarError: data not defined

In [7]:
## galerkin discretization of the eamc
using EAMC
dt = ones(length(qs))
dt[end] = 1000
tmax = length(qs)
g = @time EAMC.galerkin(qs[1:tmax], dt[1:tmax]);

┌ Info: Precompiling EAMC [d0d32b4c-83b1-4490-9109-c03fdd3b1e42]
└ @ Base loading.jl:1278


LoadError: UndefVarError: qs not defined

In [8]:
cs = EAMC.commitors(g, 900)
cc = reshape(cs, 30, 30, 24, 30, 30);

LoadError: UndefVarError: commitors not defined

In [9]:
using Plots
@manipulate for x=slider(1:30, 15, label="x"), y=slider(1:30, 15, label="y"), t=slider(1:24, 1, label="t")
heatmap(cc[:,:,t,y,x], title="Commitor von (x=$x,y=$y,t=24) zum zeitpunkt t=$t")
end

LoadError: LoadError: UndefVarError: @manipulate not defined
in expression starting at In[9]:2

In [10]:
sc = sum(cc[:,:,23,:,:], dims=[1,2]) |> x->reshape(x, 30,30)
heatmap(sc)

LoadError: UndefVarError: cc not defined

In [11]:
## restrict clustering area to the center
tcluster = 12 # timepoint at which to seperate the clusters
xmin, xmax, ymin, ymax = 1,30,1,30
M = reshape(cc[xmin:xmax, ymin:ymax, tcluster, xmin:xmax, ymin:ymax], 30*30*1, 30*30)
#css = css ./ sqrt.(sum(css, dims=1))
#M = reshape(cc[:,:,12,:,:], 30*30, 30*30)

M = M' * M
M = M - Diagonal(M)
M = M ./ sum(M, dims=2);

LoadError: UndefVarError: cc not defined

In [12]:
include("src/pccap.jl")
nc = 6
p = pccap(M, nc, optimize=true)
# now sort by cluster size
p = p[:,sortperm(sum(p, dims=1)|>vec, rev=true)];

LoadError: could not open file /Users/alex/Documents/code/generators/notebooks/src/pccap.jl

In [13]:
cl = reshape(cc[:,:,:, xmin:xmax, ymin:ymax], 30*30*24, 30*30) * p
cl = reshape(cl, 30, 30, 24, nc);

┌ Warning: /Users/alex/Documents/code/generators/notebooks/src/pccap.jl is not a file, omitting from revision tracking
└ @ Revise /Users/alex/.julia/packages/Revise/SZ4ae/src/parsing.jl:24


LoadError: UndefVarError: cc not defined

In [14]:
using Interact
gr()
@manipulate for i = 1:24, c=1:nc
heatmap(log.(cl[:,:,i,c].+1), title="pccap on center, t=$i", clims=(0,.3))
    end

@manipulate for i = 1:24, c=1:nc
heatmap(cl[:,:,i,c], title="pccap on center, t=$i")
    end

LoadError: ArgumentError: Package Interact not found in current path:
- Run `import Pkg; Pkg.add("Interact")` to install the Interact package.


In [15]:

anim = @animate for t=1:24
plot([heatmap(cl[:,:,t,c]) for c=1:nc]..., layout=grid(nc,1), size=(600,1000), title="$t", clims=(0,1))
end

gif(anim, fps=6)

LoadError: UndefVarError: nc not defined